```
Example协议块
message Example {
  Features features = 1;
};
 
message Features {
  map<string, Feature> feature = 1;
};
 
message Feature {
  oneof kind {
    BytesList bytes_list = 1;
    FloatList float_list = 2;
    Int64List int64_list = 3;
  }
```

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

#### 1. 将输入转化成TFRecord格式并保存。

In [2]:
# 定义函数转化变量类型。
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# 将数据转化为tf.train.Example格式。
def _make_example(pixels, label, image):
    image_raw = image.tostring()
    example = tf.train.Example(features=tf.train.Features(feature={
        'pixels': _int64_feature(pixels),
        'label': _int64_feature(np.argmax(label)),
        'image_raw': _bytes_feature(image_raw)
    }))
    return example

# 读取mnist训练数据。
mnist = input_data.read_data_sets("../../datasets/MNIST_data",dtype=tf.uint8, one_hot=True)
images = mnist.train.images
labels = mnist.train.labels
pixels = images.shape[1]
num_examples = mnist.train.num_examples

# 输出包含训练数据的TFRecord文件。
with tf.python_io.TFRecordWriter("output.tfrecords") as writer:
    for index in range(num_examples):
        example = _make_example(pixels, labels[index], images[index])
        writer.write(example.SerializeToString())
print("TFRecord训练文件已保存。")

# 读取mnist测试数据。
images_test = mnist.test.images
labels_test = mnist.test.labels
pixels_test = images_test.shape[1]
num_examples_test = mnist.test.num_examples

# 输出包含测试数据的TFRecord文件。
with tf.python_io.TFRecordWriter("output_test.tfrecords") as writer:
    for index in range(num_examples_test):
        example = _make_example(
            pixels_test, labels_test[index], images_test[index])
        writer.write(example.SerializeToString())
print("TFRecord测试文件已保存。")

Extracting ../../datasets/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../datasets/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data/t10k-labels-idx1-ubyte.gz
TFRecord训练文件已保存。
TFRecord测试文件已保存。


#### 2. 读取TFRecord文件

In [3]:
# 读取文件。
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(["output.tfrecords"])
_,serialized_example = reader.read(filename_queue)

# 解析读取的样例。
features = tf.parse_single_example(
    serialized_example,
    features={
        'image_raw':tf.FixedLenFeature([],tf.string),
        'pixels':tf.FixedLenFeature([],tf.int64),
        'label':tf.FixedLenFeature([],tf.int64)
    })

images = tf.decode_raw(features['image_raw'],tf.uint8)
labels = tf.cast(features['label'],tf.int32)
pixels = tf.cast(features['pixels'],tf.int32)

sess = tf.Session()

# 启动多线程处理输入数据。
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess,coord=coord)

for i in range(10):
    image, label, pixel = sess.run([images, labels, pixels])

In [ ]:

"""
读取二进制文件转换成张量，写进TFRecords,同时读取TFRcords
"""
 
#命令行参数
FLAGS = tf.app.flags.FLAGS       #获取值
tf.app.flags.DEFINE_string("tfrecord_dir","./tmp/cifar10.tfrecords","写入图片数据文件的文件名")
 
 
#读取二进制转换文件
class CifarRead(object):
    """
    读取二进制文件转换成张量，写进TFRecords,同时读取TFRcords
    """
    def __init__(self,file_list):
        """
        初始化图片参数
        :param file_list:图片的路径名称列表
        """
 
        #文件列表
        self.file_list = file_list
 
        #图片大小，二进制文件字节数
        self.height = 32
        self.width = 32
        self.channel = 3
        self.label_bytes = 1
        self.image_bytes = self.height * self.width * self.channel
        self.bytes = self.label_bytes + self.image_bytes
 
 
    def read_and_decode(self):
        """
        解析二进制文件到张量
        :return: 批处理的image,label张量
        """
        #1.构造文件队列
        file_queue = tf.train.string_input_producer(self.file_list)
 
        #2.阅读器读取内容
        reader = tf.FixedLengthRecordReader(self.bytes)
 
        key ,value = reader.read(file_queue)    #key为文件名，value为元组
 
        print(value)
 
        #3.进行解码，处理格式
        label_image = tf.decode_raw(value,tf.uint8)
        print(label_image)
 
        #处理格式，image，label
        #进行切片处理，标签值
        #tf.cast()函数是转换数据格式，此处是将label二进制数据转换成int32格式
        label = tf.cast(tf.slice(label_image,[0],[self.label_bytes]),tf.int32)
 
        #处理图片数据
        image = tf.slice(label_image,[self.label_bytes],[self.image_bytes])
        print(image)
 
        #处理图片的形状，提供给批处理
        #因为image的形状已经固定，此处形状用动态形状来改变
        image_tensor = tf.reshape(image,[self.height,self.width,self.channel])
        print(image_tensor)
 
        #批处理图片数据
        image_batch,label_batch = tf.train.batch([image_tensor,label],batch_size=10,num_threads=1,capacity=10)
 
        return image_batch,label_batch
 
    def write_to_tfrecords(self,image_batch,label_batch):
        """
        将文件写入到TFRecords文件中
        :param image_batch:
        :param label_batch:
        :return:
        """
 
        #建立TFRecords文件存储器
        writer = tf.python_io.TFRecordWriter(FLAGS.tfrecord_dir)      #传进去命令行参数
 
        #循环取出每个样本的值，构造example协议块
        for i in range(10):
 
            #取出图片的值，  #写进去的是值，而不是tensor类型，
            # 写入example需要bytes文件格式，将tensor转化为bytes用tostring()来转化
            image = image_batch[i].eval().tostring()
 
            #取出标签值，写入example中需要使用int形式，所以需要强制转换int
            label = int(label_batch[i].eval()[0])
 
            #构造每个样本的example协议块
            example = tf.train.Example(features = tf.train.Features(feature = {
                "image":tf.train.Feature(bytes_list = tf.train.BytesList(value = [image])),
                "label":tf.train.Feature(int64_list = tf.train.Int64List(value = [label]))
            }))
 
            #写进去序列化后的值
            writer.write(example.SerializeToString())     #此处其实是将其压缩成一个二进制数据
 
        writer.close()
 
        return None
 
 
 
    def read_from_tfrecords(self):
        """
        从TFRecords文件当中读取图片数据（解析example)
        :param self:
        :return: image_batch,label_batch
        """
 
        #1.构造文件队列
        file_queue = tf.train.string_input_producer([FLAGS.tfrecord_dir])    #参数为文件名列表
 
        #2.构造阅读器
        reader = tf.TFRecordReader()
 
        key,value = reader.read(file_queue)
 
        #3.解析协议块,返回的值是字典
        feature = tf.parse_single_example(value,features={
            "image":tf.FixedLenFeature([],tf.string),
            "label":tf.FixedLenFeature([],tf.int64)
        })
 
        #feature["image"],feature["label"]
        #处理标签数据    ，cast()只能在int和float之间进行转换
        label = tf.cast(feature["label"],tf.int32)    #将数据类型int64 转换为int32
 
        #处理图片数据，由于是一个string,要进行解码，  #将字节转换为数字向量表示，字节为一字符串类型的张量
        #如果之前用了tostring(),那么必须要用decode_raw()转换为最初的int类型
        # decode_raw()可以将数据从string,bytes转换为int，float类型的
        image = tf.decode_raw(feature["image"],tf.uint8)
 
        #转换图片的形状，此处需要用动态形状进行转换
        image_tensor = tf.reshape(image,[self.height,self.width,self.channel])
 
        #4.批处理
        image_batch,label_batch = tf.train.batch([image_tensor,label],batch_size=10,num_threads=1,capacity=10)
 
        return image_batch,label_batch
 
 
if __name__ == '__main__':
 
    # 找到文件路径，名字，构造路径+文件名的列表,"A.csv"...
    # os.listdir() 方法用于返回指定的文件夹包含的文件或文件夹的名字的列表
    filename = os.listdir('./data/cifar10/cifar-10-batches-bin/')
 
    #加上路径
    file_list = [os.path.join('./data/cifar10/cifar-10-batches-bin/', file) for file in filename if file[-3:] == "bin"]
 
    #初始化参数
    cr = CifarRead(file_list)
 
    #读取二进制文件
    # image_batch,label_batch = cr.read_and_decode()
 
    #从已经存储的TFRecords文件中解析出原始数据
    image_batch, label_batch = cr.read_from_tfrecords()
 
    with tf.Session() as sess:
        #线程协调器
        coord = tf.train.Coordinator()
 
        #开启线程
        threads = tf.train.start_queue_runners(sess,coord=coord)
 
        print(sess.run([image_batch,label_batch]))
 
        # print("存进TFRecords文件")
        # cr.write_to_tfrecords(image_batch,label_batch)
        # print("存进文件完毕")
 
        #回收线程
        coord.request_stop()
        coord.join(threads)